### 문제 1-2: 2단계 체인 만들기 - 영화 추천 시스템 해답

## 환경 설정

In [ ]:
# 필요한 라이브러리 설치
#%pip install -q langchain langchain-openai python-dotenv

In [4]:
# 환경 변수 설정
from dotenv import load_dotenv
import os

# .env 파일에서 API 키 로드
load_dotenv()

True

In [3]:
# 필요한 라이브러리 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

## 문제 2 해답 구현

### 요구사항
1. 1단계 체인: 장르를 입력받아 영화 1편 추천
2. 2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
3. ChatPromptTemplate 사용
4. 두 체인을 LCEL로 연결
5. 각 단계의 결과를 모두 출력하여 과정 확인

In [21]:
# 1단계: 장르별 영화 추천 프롬프트
movie_recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 전문가입니다. 사용자가 요청한 장르에 맞는 영화 1편을 추천하고 간단한 설명을 제공해주세요."),
    ("user", "{genre} 장르의 한국영화 1편을 추천해주세요. 영화 제목과 왜 이 영화를 추천하는지 이유도 함께 알려주세요.")
])

In [22]:
# 2단계: 영화 줄거리 요약 프롬프트
plot_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다. 영화의 핵심 내용을 3줄로 간결하게 요약해주세요."),
    ("user", "다음 영화 추천 내용을 바탕으로 해당 영화의 등장인물과 줄거리를 정확히 10줄로 요약해주세요: {movie}")
])

In [23]:
# LLM 모델 초기화
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 1단계 체인: 영화 추천
recommendation_chain = movie_recommendation_prompt | llm | StrOutputParser()
# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie = recommendation_chain.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화:", movie)  # movie 값 출력

# 2단계 체인: 줄거리 요약 (1단계 결과를 입력으로 받음)
plot_chain = (
    {"movie": recommendation_chain}  # 1단계 체인의 결과를 movie_info로 전달
    | plot_summary_prompt
    | llm
    | StrOutputParser()
)

 추천된 영화: **영화 제목:** '올드보이'

**추천 이유:**

'올드보이'는 2003년에 개봉한 영화로, 한국 영화계의 거장 박찬욱 감독이 연출한 작품입니다. 이 영화는 드라마 장르에 속하며, 복수, 미스터리, 심리 드라마의 요소가 혼합되어 있습니다. 주인공 오대수(최민식)는 아무런 이유도 모른 채 15년간 감금된 채로 살아오다가 풀려나 복수를 다짐합니다. 그러나 복수의 과정에서 그는 자신의 과거와 감금의 진실을 조금씩 깨닫게 됩니다. 영화는 복수에 대한 욕망과 인간의 본성, 그리고 용서에 대한 깊은 성찰을 담고 있습니다.

이 영화는 강렬한 스토리라인, 뛰어난 연기력, 그리고 감각적인 연출로 많은 찬사를 받았습니다. 특히, 최민식의 연기는 매우 인상적이며, 영화의 긴장감과 분위기를 한층 더 높여줍니다. '올드보이'는 한국 영화의 대표작 중 하나로 꼽히며, 드라마 장르를 좋아하는 분들에게 강력히 추천합니다.


In [24]:
# 테스트 실행
try:
    genre = "Drama"
    
    print("영화 추천 시스템")
    print("==================")
    print(f"\n입력 장르: {genre}")
    
    # 1단계 실행 및 결과 출력
    movie_recommendation = recommendation_chain.invoke({"genre": genre})
    print("\n1단계 - 영화 추천 결과:")
    print(movie_recommendation)
    
    print("\n---\n")
    
    # 2단계 실행 및 결과 출력
    plot_summary = plot_chain.invoke({"genre": genre})
    print("2단계 - 영화 줄거리 요약:")
    print(plot_summary)
    
except Exception as e:
    print(f"오류 발생: {e}")

영화 추천 시스템

입력 장르: Drama

1단계 - 영화 추천 결과:
**추천 영화: "올드보이(2003)"**

"올드보이(2003)"는 박찬욱 감독의 한국 드라마 영화입니다. 이 영화는 주인공 오대수(최민식)가 15년간 감금된 후, 자신의 과거와 비밀을 풀어나가는 과정을 그린 작품입니다.

이 영화를 추천하는 이유는 다음과 같습니다.

*   긴장감과 서스펜스가 넘치는 이야기: 영화는 오대수의 감금과 탈출, 그리고 복수를 향한 여정을 따라가며 긴장감과 서스펜스를 선사합니다.
*   깊은 캐릭터 심리 묘사: 영화는 각 캐릭터의 심리를 깊이 있게 탐구하며, 인간의 본성과 감정의 복잡성을 표현합니다.
*   박찬욱 감독의 스타일과 비주얼: 박찬욱 감독은 특유의 비주얼 스타일과 연출로 이 영화를 만들어냈습니다. 영화의 촬영, 편집, 그리고 음악이 모두 예술적으로 조화롭게 어우러져 있습니다.

"올드보이(2003)"는 한국 드라마 영화 중에서도 특히 깊은 감동과 생각을 불러일으키는 작품입니다. 이 영화를 통해 한국 영화의 매력을 경험해 보세요!

---

2단계 - 영화 줄거리 요약:
Here is a summary of the movie "Oldboy" in 10 lines:

*   주인공 오대수(미키)는 15년간 기억을 잃은 채로 깨어납니다.
*   그는 자신의 과거를 찾기 위해 노력합니다.
*   오대수는 자신의 납치와 장기 감금의 이유를 알기 위해 노력합니다.
*   그는 자신의 과거와 관련된 단서를 찾기 시작합니다.
*   오대수는 자신의 감금과 관련된 비밀을 발견합니다.
*   그는 자신의 복수를 위해 계획을 세웁니다.
*   오대수의 과거와 관련된 인물들이 등장합니다.
*   오대수는 자신의 기억을 회복하고 과거의 진실을 알게 됩니다.
*   영화는 반전과 결말로 유명합니다.
*   올드보이는 인간 심리와 복수라는 주제를 깊이 있게 다루고 있습니다.


## 추가 테스트

In [7]:
# 다른 장르로 추가 테스트
genre2 = "로맨스"

movie_recommendation2 = recommendation_chain.invoke({"genre": genre2})
plot_summary2 = plot_chain.invoke({"genre": genre2})

print(f"다른 장르로 테스트 - {genre2}:")
print("===============================")
print("\n영화 추천:")
print(movie_recommendation2)
print("\n줄거리 요약:")
print(plot_summary2)

다른 장르로 테스트 - 로맨스:

영화 추천:
로맨스 장르에서 라라랜드를 추천합니다!

이 영화는 데이미언 셔젤 감독이 연출한 2016년 작품으로, 라이언 고슬링과 엠마 스톤이 주연을 맡았습니다. 클래식 뮤지컬의 감성과 현대적인 로맨스가 완벽하게 조화를 이룬 작품으로, 꿈을 쫓는 두 사람의 아름답고도 애틋한 사랑 이야기를 그렸습니다.

줄거리 요약:
영화: 라라랜드

3줄 줄거리 요약:

1. 배우를 꿈꾸는 미아와 재즈 피아니스트를 꿈꾸는 세바스찬이 로스앤젤레스에서 만나 서로의 꿈을 응원하며 사랑에 빠집니다.

2. 두 사람은 각자의 꿈을 이루기 위해 노력하지만, 성공과 사랑 사이에서 선택의 기로에 서게 되며 결국 서로 다른 길을 걷게 됩니다.

3. 5년 후, 각자 꿈을 이룬 두 사람이 우연히 재회하며 함께했던 시간과 가능했을 미래를 그리워하는 애틋한 감정을 나누게 됩니다.


## 체인 연결 패턴 설명

In [ ]:
# 체인 연결 방식 설명
print("체인 연결 방식 설명:")
print("====================")
print()
print("1단계 체인:")
print("movie_recommendation_prompt | llm | StrOutputParser()")
print()
print("2단계 체인:")
print('{"movie": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()')
print()
print("핵심 포인트:")
print("- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달")
print("- movie 키를 통해 데이터 흐름 제어")
print("- LCEL을 통해 직관적인 파이프라인 구성")

체인 연결 방식 설명:

1단계 체인:
movie_recommendation_prompt | llm | StrOutputParser()

2단계 체인:
{"movie_info": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()

핵심 포인트:
- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달
- movie_info 키를 통해 데이터 흐름 제어
- LCEL을 통해 직관적인 파이프라인 구성


## 학습 정리

### 문제 2에서 배운 핵심 개념
1. **ChatPromptTemplate**: 시스템과 사용자 메시지 구조화
2. **다단계 체인**: 여러 단계로 나누어 복잡한 작업 처리
3. **체인 간 데이터 전달**: {"key": previous_chain} 패턴
4. **단계별 결과 확인**: 각 체인의 중간 결과 출력

### 구현 완료 사항
- ✅ 2단계 체인 구현
- ✅ ChatPromptTemplate 사용
- ✅ 체인 간 데이터 전달
- ✅ 영화 추천 기능
- ✅ 줄거리 요약 기능
- ✅ 각 단계별 결과 출력
- ✅ 다양한 장르 테스트

### 체인 연결의 핵심 패턴
```python
# 기본 패턴
chain1 = prompt1 | llm | parser
chain2 = {"input_key": chain1} | prompt2 | llm | parser
```

이 패턴을 통해 복잡한 작업을 단계별로 분해하여 처리할 수 있습니다.